# Multilayer ANN for Sentiment Analysis

In [1]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
import warnings
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')


df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


In [2]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

In [3]:
tokenizer('This :) is a <a> test! :-)</br>')

['test', ':)', ':)']

## Preparacion del model con Word2vect

Dimensiones utilizadas: 50, 100 y 200

In [4]:
reviewList = []
wordList = []
countRows = 0
model = np.empty((50000,51))
model[:,:] = 0
mean = np.empty((1,50))

for text in df.loc[:,'review']:
    countColumns=0
    wordList = tokenizer(text)
    reviewList.append(wordList)
    W2V = Word2Vec(wordList,size=50,min_count=1,workers=10)
    W2V.train(wordList,total_examples=len(wordList),epochs=10)
    X = W2V[W2V.wv.vocab]
    mean[0,:]  = X.mean(0)
    model[countRows,0:50] = mean[0,:]
    model[countRows,50:51] = df.loc[countRows,'sentiment']
    if (countRows % 5000 == 0) or (countRows == 0):
        print("charging: ",countRows/500,"%") 
    elif (countRows == 49999):
        print("charging:  100%");
    countRows+=1



charging:  0.0 %
charging:  10.0 %
charging:  20.0 %
charging:  30.0 %
charging:  40.0 %
charging:  50.0 %
charging:  60.0 %
charging:  70.0 %
charging:  80.0 %
charging:  90.0 %
charging:  100%


In [5]:
model.shape

(50000, 51)

In [6]:
xTrain = model[0:40000,0:50] 
yTrain = model[0:40000,50:51]
xTest = model[40000:50000,0:50]
yTest = model[40000:50000,50:51]
print(xTrain.shape)
print(yTrain.shape)
xTrain[0:20,:]

(40000, 50)
(40000, 1)


array([[ 4.56641195e-03,  8.21998622e-03,  7.71887208e-05,
        -1.76106149e-03, -8.23871326e-03, -6.96096290e-03,
         2.66548106e-03, -8.68979935e-03, -2.27937195e-03,
        -2.36079865e-03, -5.57621289e-03,  7.62806647e-03,
         1.46892539e-03, -1.58770324e-03,  1.85995406e-04,
         1.25639583e-03,  3.15499585e-03, -5.31664025e-03,
        -3.79232783e-03,  2.49201898e-03,  2.63380748e-03,
        -1.84333383e-03, -3.34703177e-03,  1.69122336e-03,
         4.24995599e-03, -2.43492448e-03, -1.82193751e-03,
        -2.17963359e-03, -2.49314588e-03,  3.17268353e-03,
        -3.62678571e-03,  1.09000294e-03, -1.97822740e-03,
         2.41163583e-03,  3.07962345e-03, -3.69907299e-04,
        -2.18141417e-04,  1.07424590e-03,  9.67114815e-04,
        -2.43674917e-03, -3.23110027e-03, -1.35833793e-03,
        -2.69482587e-03,  2.37985048e-03, -5.47054084e-03,
        -3.66479182e-03, -3.76766338e-03,  2.50080018e-03,
        -4.65084752e-03, -5.66482358e-03],
       [ 2.57

## Implementacion de Multilayer ANN 

Numero de capas experimentadas: 2, 3, 5, 10 
Numero de neuronas experimentadas: 2, 3, 5, 10
Funcion de activacion: Funcion Sigmoidal

In [15]:
numberCycle = 5000
numberHiddenLayer = 3
numberNeuron = 3
numberLayer = numberHiddenLayer + 1


#alfa[0,0] = np.power((1+np.power(5,0.5))/2,-1)
b = np.random.rand(numberNeuron,numberHiddenLayer)
#b[:,:] = 1
bOut = np.random.rand(1,1)
#bOut[:,:] = 1

lamba = np.random.rand(1,1)
lamba[0,0] = np.power(10.0,-8.0)
wInput = np.random.rand(numberNeuron,50) 
wHidden1 = np.random.rand(numberNeuron,numberNeuron) 
wHidden2 = np.random.rand(numberNeuron,numberNeuron)
wOutput =  np.random.rand(numberNeuron,1) 

wInput[:,:] = np.random.rand()
#wHidden[:,:] = 0.1*np.random.rand()
wOutput[:,:] = np.random.rand()

derivateInput = np.empty((numberNeuron,50))
derivate1 = np.empty((numberNeuron,numberNeuron))
derivate2 = np.empty((numberNeuron,numberNeuron))
derivate = 0

wCurrent = np.zeros((numberNeuron,numberNeuron))
yHidden1 = np.zeros((numberNeuron,1))
yHidden2 = np.zeros((numberNeuron,1))
yHidden3 = np.zeros((numberNeuron,1))
yOutput = 0
E = np.zeros((numberNeuron,numberHiddenLayer))

hHidden1 = np.zeros((numberNeuron,1))
hHidden2 = np.zeros((numberNeuron,1))
hHidden3 = np.zeros((numberNeuron,1))
hOutput = np.zeros((1,1))
e = np.zeros((1,1))

Y = np.zeros((40000,1))
Ypredict = np.zeros((10000,1))
def sigmoid(MU):
    return 1 / (1 + np.exp(-MU))
    
def runANN():
    for cycle in range (0, numberCycle):
        for i in range(0,40000):
            y = forward(numberLayer,xTrain[i,:])
            Y[i,0] = y
            #print("out ",y)
            e[0,0] = getError(y,yTrain[i,0])
            #print("e ", e)
            backpropagation()
            updateWeight(i)
        if (cycle % 10 == 0):
            for j in range(0,10000):
                predictY = forward(numberLayer,xTest[j,:])
                Ypredict[j,0] = predictY
            getAccuracy(cycle,yTest,Ypredict)
            print(Ypredict)
            
def forward(numberLayer,Data):
    for hl in range (0,numberLayer):
        if(hl == 0):
            hHidden1[:,0] = np.dot(wInput[:,:],Data.T) + b[:,hl]
            yHidden1[:,0] = sigmoid(hHidden1[:,0])
            #print(wInput[:,:]) 
        elif(hl == 1):
            hHidden2[:,0] = np.dot(wHidden1[:,:],yHidden1[:,0]) + b[:,hl]
            yHidden2[:,0] = sigmoid(hHidden2[:,0])
            #print("pesos ", wCurrent[:,:])
            #print(wHidden1[:,:]) 
        elif(hl==2):
            hHidden3[:,0] = np.dot(wHidden2[:,:],yHidden2[:,0]) + b[:,hl]
            yHidden3[:,0] = sigmoid(hHidden3[hl-1,:])
            #print(wHidden2[:,:]) 
        else: 
            hOutput[0,0] = np.dot(wOutput[:,0],yHidden3[:,0].T) + bOut[0,0]
            yOutput = sigmoid(hOutput[0,0])
            #print("pesos", wOutput[:,0])
            
            #print("out",yOutput) 
    return yOutput

def getError(y,yTrain):
    e =  y - yTrain
    #print(e)
    return e
     
def backpropagation():
    for hl in range (numberLayer-1,0,-1):
        #if(hl == 0):
        #    E[hl,:] = np.dot(wInput[:,:].T,E[:,hl-1]) 
        if(hl == 1):
            sumError = E[:,hl].sum()
            E[:,hl-1] = sumError * (yHidden1[:,0]*(1-yHidden1[:,0]))
            #print("whiden ", wHidden1[:,:])
            #print("error ", E[:,hl])
            #print("y ", yHidden[hl-1,:]*(1-yHidden[hl-1,:]))
            #print("hl ", hl, " e ",E[:,hl-1])
        if(hl == 2):
            sumError = E[:,hl].sum()
            E[:,hl-1] = sumError * (yHidden2[:,0]*(1-yHidden2[:,0]))
            #print("whiden ", wHidden2[:,:])
            #print("error ", E[:,hl])
            #print("y ", yHidden[hl-1,:]*(1-yHidden[hl-1,:]))
            #print("hl ", hl, " e ",E[:,hl-1])
        else:
            E[:,hl-1] = np.dot(yHidden3[:,0]*(1-yHidden3[:,0]),e[0,0])
            #E[:,hl-1] = np.dot(wOutput[:,0],e[0,0])
            #print("whiden ", wOutput[:,0])
            #print("error ", e[0,0])
            #print("y  ", yHidden[hl-1,:]*(1-yHidden[hl-1,:]))
            #print("hl ", hl, " e ",E[:,hl-1])
            
def updateWeight(i):
    alfa = np.power(10.0,-1.0) 
    for hl in range (0,numberLayer):
        if(hl == 0):
            #E[:,hl] = wHidden1[:,:].T * E[:,hl] * (yHidden[hl-1,:]*(1-yHidden[hl-1,:]))
            derivateInput[:,:] = np.dot(E[:,hl].reshape((3,1)),xTrain[i,:].reshape((1,50))) 
            wInput[:,:] = ((1-lamba * alfa) * wInput[:,:]) - alfa * derivateInput[:,:]
            #print("derivate ",derivateInput)
            #print("menor ",hl)
        elif(hl==1):
            derivate1[:,:] = np.dot(E[:,hl],yHidden1[:,0].T)  
            #print(wHidden[:,:,hl-1])
            wHidden1[:,:] = ((1-lamba * alfa) * wHidden1[:,:])  - alfa * derivate1[:,:]
         #  b[hl-1,:] -=  alfa[0,0] * E[:,hl].T
        elif(hl == 2):
            derivate2[:,:] = np.dot(E[:,hl],yHidden2[:,0].T)  
            #print(wHidden[:,:,hl-1])
            wHidden2[:,:] = ((1-lamba * alfa) * wHidden2[:,:]) - alfa * derivate2[:,:]
       #     b[hl-1,:] -=  alfa[0,0] * E[:,hl].T
            #print(wHidden[:,:,hl-1])
            #print("menor ",hl)
        else:
            #print(wOutput[:,0])
            derivate = np.dot(e[0,0],yHidden3[:,0])
            wOutput[:,0] = ((1-lamba * alfa) * wOutput[:,0]) - alfa*derivate
            bOut[0,0] -= alfa * e[0,0]
            #print(wOutput[:,0])
            #print("igual ",hl)
def getAccuracy(i,yTest,predictTest):
    print("accuracy in the cycle: ",i, " is: ",accuracy_score(yTest, predictTest.round()))


In [16]:
runANN()

accuracy in the cycle:  0  is:  0.5013
[[0.38977344]
 [0.38977349]
 [0.38977378]
 ...
 [0.38977367]
 [0.38977328]
 [0.3897737 ]]
accuracy in the cycle:  10  is:  0.5013
[[0.38947917]
 [0.38947924]
 [0.38947773]
 ...
 [0.3894791 ]
 [0.38947882]
 [0.38947952]]
accuracy in the cycle:  20  is:  0.5013
[[0.38949415]
 [0.38949426]
 [0.38949143]
 ...
 [0.38949393]
 [0.38949358]
 [0.38949467]]


KeyboardInterrupt: 

In [13]:
np.savetxt("model50dim.csv", model, delimiter=",")